In [ ]:
import RPi.GPIO as GPIO
import gpiozero
import time
import numpy as np

In [ ]:
# Constants
# Encoder pins
# ML = motor left, MR = motor right
ML_ENC_A = 2 # yellow encoder
ML_ENC_B = 3 # white encoder 

MR_ENC_A = 14 # yellow encoder
MR_ENC_B = 15 # white encoder

"""
- gear ratio = 75:1 = motor shaft rotates 75 times each revolution
- encoder = 48 CPR (count per revolution) = 48 counts each shaft rotation
- counts per round = count per rev * rev per round = 48*75 = 3600
"""
MAX_ENC_STEPS = 3600

# Motor Pins
ML_IN1 = 17 # IN1
ML_IN2 = 27 # IN2
ML_EN = 11 # brown

MR_IN1 = 10 # IN3
MR_IN2 = 22 # IN4
MR_EN = 9



In [ ]:
# import RPi.GPIO as GPIO
# import gpiozero
# import time
# import numpy as np

# Physical dimensions
WHEEL_RADIUS = 0.05 # meters
WHEEL_SEP = 0.222 # meters
# wheel separation measurement (ASCII art)
"""
    <------- l ------->
    |                 |
 ___|___           ___|___
|       |         |       |
|   O   |         |   O   |
|_______|         |_______|
    ^                 ^
    |                 |
  Left wheel       Right wheel
  center            center
"""

class DiffDriveRobot:
    def __init__(self, dt=0.1, wheel_radius=WHEEL_RADIUS, wheel_sep=WHEEL_SEP):
        self.x = 0.0  # x-position
        self.y = 0.0  # y-position 
        self.th = 0.0  # orientation
        
        self.wl = 0.0  # rotational velocity left wheel
        self.wr = 0.0  # rotational velocity right wheel
        
        self.dt = dt # time delta in seconds
        self.r = wheel_radius # wheel radius in meters
        self.l = wheel_sep # wheel separation in meters

        # Initialize encoders
        self.ML_ENC = gpiozero.RotaryEncoder(a=ML_ENC_A, b=ML_ENC_B, max_steps=MAX_ENC_STEPS, wrap=True)
        self.MR_ENC = gpiozero.RotaryEncoder(a=MR_ENC_A, b=MR_ENC_B, max_steps=MAX_ENC_STEPS, wrap=True)

        # Initialize motor control pins
        self.ML_PWM = GPIO.PWM(ML_EN, 1000)  # 1000 Hz frequency
        self.MR_PWM = GPIO.PWM(MR_EN, 1000)
        self.ML_PWM.start(0)
        self.MR_PWM.start(0)

    def read_encoder(self, encoder):
        return encoder.steps / MAX_ENC_STEPS * 2 * np.pi  # Convert steps to radians

    def set_motor_speed(self, left_duty_cycle, right_duty_cycle):
        # Set direction
        GPIO.output(ML_IN1, GPIO.HIGH if left_duty_cycle >= 0 else GPIO.LOW)
        GPIO.output(ML_IN2, GPIO.LOW if left_duty_cycle >= 0 else GPIO.HIGH)
        GPIO.output(MR_IN1, GPIO.HIGH if right_duty_cycle >= 0 else GPIO.LOW)
        GPIO.output(MR_IN2, GPIO.LOW if right_duty_cycle >= 0 else GPIO.HIGH)

        # Set speed
        self.ML_PWM.ChangeDutyCycle(abs(left_duty_cycle) * 100)
        self.MR_PWM.ChangeDutyCycle(abs(right_duty_cycle) * 100)

    def base_velocity(self, wl, wr):
        v = (wl * self.r + wr * self.r) / 2.0
        w = (wl * self.r - wr*self.r)/self.l
        return v, w

    def pose_update(self, duty_cycle_l, duty_cycle_r):
        self.set_motor_speed(duty_cycle_l, duty_cycle_r)
        
        # Read actual wheel velocities from encoders
        wl_prev = self.wl
        wr_prev = self.wr
        self.wl = (self.read_encoder(self.ML_ENC) - wl_prev) / self.dt
        self.wr = (self.read_encoder(self.MR_ENC) - wr_prev) / self.dt
        
        v, w = self.base_velocity(self.wl, self.wr)
        
        self.x = self.x + self.dt * v * np.cos(self.th)
        self.y = self.y + self.dt * v * np.sin(self.th)
        self.th = self.th + w * self.dt
        
        return self.x, self.y, self.th

class RobotController:
    def __init__(self, Kp=0.1, Ki=0.01, wheel_radius=WHEEL_RADIUS, wheel_sep=WHEEL_SEP):
        self.Kp = Kp # proportional gain
        self.Ki = Ki # integral gain
        self.r = wheel_radius
        self.l = wheel_sep
        self.e_sum_l = 0
        self.e_sum_r = 0
        
    def p_control(self, w_desired, w_measured, e_sum):
        duty_cycle = min(max(-1, self.Kp * (w_desired - w_measured) + self.Ki * e_sum), 1)
        e_sum = e_sum + (w_desired - w_measured)
        return duty_cycle, e_sum
        
    def drive(self, v_desired, w_desired, wl, wr):
        wl_desired = (v_desired + self.l * w_desired / 2) / self.r
        wr_desired = (v_desired - self.l * w_desired / 2) / self.r
        
        duty_cycle_l, self.e_sum_l = self.p_control(wl_desired, wl, self.e_sum_l)
        duty_cycle_r, self.e_sum_r = self.p_control(wr_desired, wr, self.e_sum_r)
        
        return duty_cycle_l, duty_cycle_r

# # Set GPIO modes
# GPIO.setmode(GPIO.BCM)
# GPIO.setup(ML_IN1, GPIO.OUT)
# GPIO.setup(ML_IN2, GPIO.OUT)
# GPIO.setup(MR_IN1, GPIO.OUT)
# GPIO.setup(MR_IN2, GPIO.OUT)
# GPIO.setup(ML_EN, GPIO.OUT)
# GPIO.setup(MR_EN, GPIO.OUT)
# 
# # https://gpiozero.readthedocs.io/en/stable/api_input.html#rotaryencoder
# ML_ENC = gpiozero.RotaryEncoder(a=ML_ENC_A, b=ML_ENC_B, max_steps=MAX_ENC_STEPS, wrap=True) 
# MR_ENC = gpiozero.RotaryEncoder(a=MR_ENC_A, b=MR_ENC_B, max_steps=MAX_ENC_STEPS, wrap=True) 

# Set GPIO modes
GPIO.setmode(GPIO.BCM)
GPIO.setup(ML_IN1, GPIO.OUT)
GPIO.setup(ML_IN2, GPIO.OUT)
GPIO.setup(MR_IN1, GPIO.OUT)
GPIO.setup(MR_IN2, GPIO.OUT)
GPIO.setup(ML_EN, GPIO.OUT)
GPIO.setup(MR_EN, GPIO.OUT)

# Main control loop
robot = DiffDriveRobot()
controller = RobotController()

try:
    while True:
        # Example: Move forward at 0.5 m/s with no rotation
        v_desired = 0.5
        w_desired = 0.0
        
        duty_cycle_l, duty_cycle_r = controller.drive(v_desired, w_desired, robot.wl, robot.wr)
        x, y, th = robot.pose_update(duty_cycle_l, duty_cycle_r)
        
        print(f"Position: ({x:.2f}, {y:.2f}), Orientation: {th:.2f}")
        time.sleep(robot.dt)

except KeyboardInterrupt:
    print("Stopping robot")
    robot.set_motor_speed(0, 0)
    GPIO.cleanup()